In [2]:
import pandas as pd

In [5]:
movies = pd.read_csv('../data/movies.csv')
ratings = pd.read_csv('../data/ratings.csv')

In [6]:
print("\n***Movies***\n")
movies.head()


***Movies***



movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [7]:
print("\n***Ratings***\n")
ratings.head()


***Ratings***



userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

In [8]:
num_movies = movies['movieId'].nunique()
num_users = ratings['userId'].nunique()

In [9]:
print(f"Number of Movies: {num_movies}")
print(f"\nNumber of Users: {num_users}")

Number of Movies: 9742

Number of Users: 610


## üìÖ **Day 2: Data Preprocessing for Collaborative Filtering**

### üéØ **Main Goal**:
Prepare your MovieLens dataset for collaborative filtering by:
- Cleaning the data (handle missing values, filter sparse/low-rated data).
- Creating a **User-Movie Ratings Matrix**, which will later be used to compute similarities.

---

### ‚úÖ **Outcome at the end of Day 2**:
- A **cleaned dataset** (filtered to remove noise or insufficient data).
- A **User-Movie Ratings Matrix** (users as rows, movies as columns, ratings as values).
- You're now ready to calculate similarities from this matrix in later days.

---

## üîÅ Step-by-Step Plan

### **1. Import Necessary Libraries**

### **2. Load the Ratings CSV**

### **3. Check for Missing Values**

#### üß† **Why?**
Missing values can cause issues when building the user-movie matrix.

### **4. Filter Out Infrequently Rated Movies**

#### üß† **Why?**
Movies with very few ratings give weak collaborative filtering signals and make the similarity matrix sparse.

#### üëá Here's the **explanation** of the code 

```python
# Count how many times each movie has been rated
movies_count = ratings['movieId'].value_counts()
```

- üîç `value_counts()` ‚Üí Think of it as: ‚ÄúHow many times does each movie appear in the ratings?‚Äù

```python
# Keep only movies with at least 10 ratings

# Day-2

# Documentation Concepts to Read Up On:
| Concept | What to Search |
|--------|----------------|
| `pandas.DataFrame.value_counts()` | Count frequencies of values |
| `pandas.Series.isin()` | Filter based on a list of values |
| `pandas.pivot_table()` | Creating pivot (matrix-style) tables |
| Missing values (`NaN`) | `dropna()`, handling NaNs |
| Filtering DataFrames | Boolean indexing in Pandas |

---

In [11]:
ratings.isnull()

userId  movieId  rating  timestamp
0        False    False   False      False
1        False    False   False      False
2        False    False   False      False
3        False    False   False      False
4        False    False   False      False
...        ...      ...     ...        ...
100831   False    False   False      False
100832   False    False   False      False
100833   False    False   False      False
100834   False    False   False      False
100835   False    False   False      False

[100836 rows x 4 columns]

In [24]:
# Counting how many times each movies has been rated
movies_count = ratings['movieId'].value_counts()
movies_count

movieId
356       329
318       317
296       307
593       279
2571      278
         ... 
188833      1
189381      1
3899        1
2848        1
147002      1
Name: count, Length: 9724, dtype: int64

In [28]:
# Keeping only moviews with at least 10 ratings
popular_movies = movies_count[movies_count >= 10].index
popular_movies

Index([   356,    318,    296,    593,   2571,    260,    480,    110,    589,
          527,
       ...
          258,   1290,   5621,    918,   2380,   4167,  50794,   4255,   1147,
       120466],
      dtype='int64', name='movieId', length=2269)

In [29]:
# filter the original dataset to include only those popular movies
filter_ratings = ratings[ratings['movieId'].isin(popular_movies)]
filter_ratings

userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100818     610   159093     3.0  1493847704
100829     610   164179     5.0  1493845631
100830     610   166528     4.0  1493879365
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352

[81116 rows x 4 columns]

In [31]:
# Filtering Users who rated only few Movies
users_counts = filter_ratings['userId'].value_counts()
actice_users = users_counts[users_counts >5].index
filter_ratings = filter_ratings[filter_ratings['userId'].isin(actice_users)]
filter_ratings

userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100818     610   159093     3.0  1493847704
100829     610   164179     5.0  1493845631
100830     610   166528     4.0  1493879365
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352

[81116 rows x 4 columns]

In [32]:
# Creating the user movie ratings Matrix
user_movie_matrix = filter_ratings.pivot_table(index='userId', columns='movieId', values='rating')

In [33]:
user_movie_matrix.shape

(610, 2269)

In [34]:
user_movie_matrix.head()

movieId  1       2       3       5       6       7       9       10      \
userId                                                                    
1           4.0     NaN     4.0     NaN     4.0     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  11      12      ...  166461  166528  166643  168250  168252  174055  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  176371  177765  179819  187593  
userId                                   
1           NaN     NaN     NaN     NaN  
2           NaN     NaN     NaN     NaN  
3           NaN     NaN     NaN     NaN  
4           NaN     NaN     NaN     NaN  
5           NaN     NaN     NaN     NaN  

[5 rows x 2269 columns]

# DAY 3

## What is Cosine Similarity?
- Cosine similarity measures how **‚Äúsimilar‚Äù** two vectors are by calculating the cosine of the angle between them. In simple terms, it tells you if two users rate movies in a similar way.
- **Range:** 0 to 1 (for non-negative ratings like 1‚Äì5 stars):
    - 1 = Identical preferences (users rate movies the same way).
    - 0 = No similarity (completely different tastes).


**Analogy**: Imagine two people pointing at stars in the sky. If their arms point in nearly the same direction (small angle), they‚Äôre looking at similar stars (high cosine similarity). If their arms point in opposite directions (large angle), they‚Äôre looking at different stars (low similarity).

**In AI**: Cosine similarity is used in recommendation systems (like Netflix) to find similar users or items, powering collaborative filtering (recommending based on user behavior).

In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv("../data/movies.csv")
ratings = pd.read_csv("../data/ratings.csv")

In [6]:
# Creating USer-movie matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Fill NaN (no rating) with 0
user_movie_matrix = user_movie_matrix.fillna(0)
user_movie_matrix.head()

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  193583  193585  193587  193609  
userId                                   
1           0.0     0.0     0.0     0.0  
2           0.0     0.0     0.0     0.0  
3           0.0     0.0     0.0     0.0  
4           0.0     0.0     0.0     0.0  
5           0.0     0.0     0.0     0.0  

[5 rows x 9724 columns]

Each row is a user‚Äôs rating vector (e.g., User 1: [4.0, 0.0, 4.5, ...]).

In **AI:** This matrix is the foundation of collaborative filtering. It‚Äôs sparse (lots of zeros) because users rate only a few movies, which is typical in recommendation systems.

## The Cosine Similarity Function
Cosine similarity between two vectors ùê¥ and B is:
- coisine_similarity(A, B) = ( A . B ) / ( ||A|| ||B|| )

- **A‚ãÖB:** Dot product (sum of element-wise products).
- **‚à•ùê¥‚à•:** Magnitude (square root of sum of squared elements).
- NumPy makes this easy with vectorized operations.

In [12]:
def coisine_similarity(user1, user2):
    """
    Compute cosine similarity between two users' rating vectors.
    Parameters:
        user1, user2: NumPy arrays of ratings (e.g., user_movie_matrix.loc[1].values)
    Returns:
        Float between 0 and 1 (1 = identical, 0 = no similarity)
    """
    # Compute dot product
    dot_product = np.dot(user1, user2)

    # Compute magnitude 
    magnitude1 = np.sqrt(np.sum(user1 ** 2))
    magnitude2 = np.sqrt(np.sum(user2 ** 2))

    # Avoid division by zero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    # Compute cosine similarity
    return dot_product / (magnitude1 + magnitude2)


**What‚Äôs Happening?**
- `np.dot(user1, user2)`: Multiplies corresponding ratings and sums them (like \( 4.0*0.0 + 0.0*3.0 + ... \)).
- `np.sqrt(np.sum(user1 ** 2))`: Computes the magnitude of `user1` (square root of sum of squared ratings).
- If either user has all zeros (no ratings), return 0 to avoid division by zero.
- The result is a number between 0 and 1.

**Analogy**: Think of two users as archers shooting arrows (ratings) at a target (movies). Cosine similarity measures how close their arrows land to each other‚Äôs direction, ignoring how far they shot.

#### Step 3: Test the Function
Let‚Äôs compute the similarity between two users (e.g., `userId` 1 and 2).


In [15]:
# Get ranking vectors for Two users
user1_ratings = user_movie_matrix.loc[1].values
user2_ratings = user_movie_matrix.loc[2].values

# Compute cosine similarity
similarity = coisine_similarity(user1_ratings, user2_ratings)
print(f"Coisine Similarity between User 1 and User 2: {similarity}")

Coisine Similarity between User 1 and User 2: 0.44795632538769775


- A low score (e.g., 0.447) means User 1 and User 2 have different tastes.
- A high score (e.g., 0.9) would mean they rate movies similarly.

**What‚Äôs Happening?**
- `user_movie_matrix.loc[1].values`: Gets User 1‚Äôs ratings as a NumPy array (e.g., `[4.0, 0.0, 4.5, ...]`).
- The function computes how aligned their ratings are.
- **In AI**: A high similarity score means User 2‚Äôs favorite movies could be recommended to User 1.

#### Step 4: Apply to Recommendation System
To make recommendations:
1. Find users similar to a target user (high cosine similarity).
2. Recommend movies they rated highly that the target user hasn‚Äôt seen.


In [20]:
# Merge ratings and movies on movieId
merged_data = pd.merge(ratings, movies, on='movieId')

In [22]:
# Find movies User 2 rated highly (e.g., > 3) that User 1 hasn't rated
user2_high_rated = merged_data[(merged_data['userId'] == 2) & (merged_data['rating'] > 3)][['movieId', 'title', 'rating']]
user1_rated = merged_data[merged_data['userId'] == 1]['movieId']

# Recommend movies User 2 rated but User 1 hasn't
recommendations = user2_high_rated[~user2_high_rated['movieId'].isin(user1_rated)]
print("Recommended movies for User 1 based on User 2:")
print(recommendations)

Recommended movies for User 1 based on User 2:
     movieId                                              title  rating
234     1704                           Good Will Hunting (1997)     4.5
236     6874                           Kill Bill: Vol. 1 (2003)     4.0
237     8798                                  Collateral (2004)     3.5
238    46970  Talladega Nights: The Ballad of Ricky Bobby (2...     4.0
239    48516                               Departed, The (2006)     4.0
240    58559                            Dark Knight, The (2008)     4.5
241    60756                               Step Brothers (2008)     5.0
242    68157                        Inglourious Basterds (2009)     4.5
244    74458                              Shutter Island (2010)     4.0
246    79132                                   Inception (2010)     4.0
247    80489                                   Town, The (2010)     4.5
248    80906                                  Inside Job (2010)     5.0
249    86345     

**Note**: This is a basic example. In a full system, you‚Äôd compute similarities for all users and pick the most similar ones.

**In AI**: This is **user-based collaborative filtering**. You‚Äôre using similarity scores to find ‚Äúneighbors‚Äù (similar users) and recommend their favorite movies.

#### Troubleshooting
- **KeyError: 1**: If `userId` 1 or 2 isn‚Äôt in `user_movie_matrix`, check `user_movie_matrix.index` and pick valid `userId`s (e.g., `user_movie_matrix.index[0]`).
- **NaN in Matrix**: Ensure `fillna(0)` was applied to replace missing ratings.
- **Low Similarity Scores**: If all scores are near 0, it‚Äôs normal for sparse data (users rate few movies). Try users with more ratings or a smaller dataset.
- **File Path Issues**: Use absolute paths (e.g., `'C:/AI_Projects/movies.csv'`) and check with `!dir` (Windows) or `!ls` (Mac/Linux).

---

# Day4: Build user-based recommendation logic.
**Today Goal**: Build a function that recommends movies to a user by finding other users with similar tastes (using cosine similarity from Day 3) and suggesting movies they rated highly that the target user hasn‚Äôt seen.
**What is User-Based Recommendation Logic?**
- This is a type of **user-based collaborative filtering**, where recommendations are made by:
    - Identifying users similar to the target user (based on their rating patterns).
    - Aggregating the favorite movies of those similar users.
    - Recommending movies the target user hasn‚Äôt rated.
- **Cosine Similarity:** You‚Äôll use your Day 3 cosine similarity function to measure how similar two users‚Äô ratings are.
- **Matrices:** The user-movie rating matrix (from Day 3 and 4) will store ratings, and matrix operations (like dot products) will help compute similarities.
- **In Your Project:** You‚Äôll use ratings.csv to build the user-movie matrix and movies.csv to get movie titles and genres for recommendations.

**Analogy:** Imagine you‚Äôre at a movie club. You want to watch a new movie, so you ask people who like the same movies as you (similar users). They suggest their favorites that you haven‚Äôt seen yet. The recommendation function is like the club organizer who finds these people and compiles their suggestions.

**In AI:** User-based collaborative filtering is widely used in recommendation systems (e.g., Netflix, Amazon) to personalize suggestions based on user behavior.

### Reviewing Your Datasets
You‚Äôre working with:
1. **movies.csv** (columns: `movieId`, `title`, `genres`):
   - Example:
     ```
     movieId,title,genres
     1,Toy Story (1995),Animation|Comedy
     2,Jumanji (1995),Adventure
     3,The Matrix (1999),Action|Sci-Fi
     ```
   - Purpose: Provides movie details for recommendations (titles and genres).
2. **ratings.csv** (columns: `userId`, `movieId`, `rating`, `timestamp`):
   - Example:
     ```
     userId,movieId,rating,timestamp
     1,1,4.0,964982703
     1,3,4.5,964981247
     2,2,3.0,964982224
     ```
   - Purpose: Contains user ratings to compute similarities and find high-rated movies.

**Key Data Structure**:
- **User-Movie Matrix**: A matrix where rows are `userId`, columns are `movieId`, and values are `rating` (0 for unrated movies). This was created on Day 3:
  ```python
  user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
  ```

---

### Learning User-Based Collaborative Filtering
Let‚Äôs break down the concepts you need to understand to build the recommendation function.

#### What is User-Based Collaborative Filtering?
- It‚Äôs a method to recommend items (movies) by finding users with similar preferences and suggesting their highly rated items.
- **Steps**:
  1. **Build the User-Movie Matrix**: Represent ratings as a matrix (rows = users, columns = movies).
  2. **Compute Similarities**: Use cosine similarity to find users with similar rating patterns.
  3. **Select Top Similar Users**: Pick the top \( k \) most similar users (e.g., \( k = 5 \)).
  4. **Recommend Movies**: Suggest movies these users rated highly (e.g., > 3.5) that the target user hasn‚Äôt rated.
- **Why It Works**: If User A and User B rate movies similarly (e.g., both love sci-fi), User B‚Äôs favorite movies are likely good recommendations for User A.

**Analogy**: Think of a group of friends who enjoy similar foods. If you like spicy tacos and your friend does too, you‚Äôll probably like their favorite spicy burrito that you haven‚Äôt tried yet.

#### Role of Cosine Similarity (Day 3)
- You already wrote a cosine similarity function to measure how similar two users‚Äô rating vectors are:
  ```python
  def cosine_similarity(user1, user2):
      dot_product = np.dot(user1, user2)
      magnitude1 = np.sqrt(np.sum(user1 ** 2))
      magnitude2 = np.sqrt(np.sum(user2 ** 2))
      if magnitude1 == 0 or magnitude2 == 0:
          return 0.0
      return dot_product / (magnitude1 * magnitude2)
  ```
- This function takes two rating vectors (e.g., `[4.0, 0, 4.5]` and `[0, 3.0, 0]`) and returns a score between 0 and 1 (1 = identical tastes, 0 = no similarity).
- **In AI**: Cosine similarity is used to find ‚Äúneighbors‚Äù (similar users) in collaborative filtering.

#### Role of Matrices (Day 4)
- The user-movie matrix is a matrix where each row is a user‚Äôs ratings across all movies.
- **Matrix Operations**:
  - **Dot Product**: Used in cosine similarity (via `np.dot`) to compare rating vectors.
  - **Matrix Multiplication**: Could be used in advanced methods like matrix factorization (Day 4 extra learning), but for now, we‚Äôll focus on dot products for similarity.
- **Connection**: The user-movie matrix is sparse (many zeros for unrated movies), and cosine similarity helps navigate this sparsity by focusing on rated movies.

#### Role of Binary Search (Day 5, Optional)
- While not directly required for this project, binary search could optimize tasks like finding a specific `movieId` or `userId` in a sorted list (e.g., if you sort `movies.csv` by `movieId`).
- Example: After computing similarities, you might use binary search to quickly find a movie in a sorted list of recommended `movieId`s.

---

### Step-by-Step Guide to Build the Recommendation Function
Let‚Äôs write a function to recommend movies for a user based on similar users‚Äô ratings. We‚Äôll use **Pandas** for data manipulation, **NumPy** for cosine similarity, and your CSVs for data. I‚Äôll explain each step, provide code, and connect it to your project.


In [4]:
# loading Data
import pandas as pd
import numpy as np

# loads csv
movies = pd.read_csv("../data/movies.csv")
ratings = pd.read_csv("../data/ratings.csv")

#Create user movie matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_movie_matrix.head()

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  193583  193585  193587  193609  
userId                                   
1           0.0     0.0     0.0     0.0  
2           0.0     0.0     0.0     0.0  
3           0.0     0.0     0.0     0.0  
4           0.0     0.0     0.0     0.0  
5           0.0     0.0     0.0     0.0  

[5 rows x 9724 columns]

**What‚Äôs Happening?**
- `pivot_table`: Creates a matrix where rows are `userId`, columns are `movieId`, and values are `rating`.
- `fillna(0)`: Replaces unrated movies with 0.

**AI**: This matrix is core data structure for collaborating filtering, representing user preferences

In [5]:
# Reuse Cosine Similarity Function
def coisine_similarity(user1, user2):
    dot_product = np.dot(user1, user2)
    magnitude1 = np.sqrt(np.sum(user1 ** 2))
    magnitude2 = np.sqrt(np.sum(user2 ** 2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    return dot_product / (magnitude1 * magnitude2)

**Why?**:
- This function computes the similarity between two users‚Äô rating vectors (rows in the user-movie matrix).
- Example: If User 1‚Äôs ratings are `[4.0, 0, 4.5]` and User 2‚Äôs are `[0, 3.0, 0]`, a low similarity score indicates different tastes.

### Write the Recommendation Function
The function will:
- Compute cosine similarity between the target user and all other users.
- Select the top \( k \) most similar users.
- Recommend movies they rated highly (e.g., > 3.5) that the target user hasn‚Äôt rated.
- Return movie titles and genres from `movies.csv`.


In [10]:
def recommend_movies(user_id, user_movie_matrix, ratings_df, movies_df, k=5, rating_threshold=3.5):
    """
    Recommend movies for a user based on similar users' ratings.
    Parameters:
        user_id: ID of the target user
        user_movie_matrix: Pandas DataFrame (users x movies, ratings as values)
        ratings_df: Original ratings DataFrame (userId, movieId, rating, timestamp)
        movies_df: Original movies DataFrame (movieId, title, genres)
        k: Number of similar users to consider
        rating_threshold: Minimum rating for recommended movies
    Returns:
        DataFrame with recommended movie titles and genres
    """
    # Get target user's ratings
    if user_id not in user_movie_matrix.index:
        return pd.DataFrame(columns=['title', 'genres'])
    target_ratings = user_movie_matrix.loc[user_id].values
    
    # Compute similarities with all other users
    similarities = []
    for other_user in user_movie_matrix.index:
        if other_user != user_id:
            other_ratings = user_movie_matrix.loc[other_user].values
            sim = coisine_similarity(target_ratings, other_ratings)
            similarities.append((other_user, sim))
    
    # Sort by similarity and pick top k users
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_k_users = [user for user, _ in similarities[:k]]
    
    # Get movies rated highly by top k users
    high_rated = ratings_df[
        (ratings_df['userId'].isin(top_k_users)) & 
        (ratings_df['rating'] > rating_threshold)
    ][['movieId', 'rating']]
    
    # Filter out movies the target user has rated
    target_rated = ratings_df[ratings_df['userId'] == user_id]['movieId']
    recommendations = high_rated[~high_rated['movieId'].isin(target_rated)]
    
    # Merge with movies_df to get titles and genres
    recommendations = recommendations.merge(movies_df[['movieId', 'title', 'genres']], on='movieId')
    
    # Return unique recommendations (up to 5)
    return recommendations[['title', 'genres']].drop_duplicates().head(5)

In [11]:
# Test the recommendation function
user_id = 1
recommendations = recommend_movies(user_id, user_movie_matrix, ratings, movies, k=5, rating_threshold=3.5)
print(f"Recommended movies for User {user_id}:")
print(recommendations)

Recommended movies for User 1:
                                        title                         genres
0   Twelve Monkeys (a.k.a. 12 Monkeys) (1995)        Mystery|Sci-Fi|Thriller
1                               Friday (1995)                         Comedy
2  Rumble in the Bronx (Hont faan kui) (1995)  Action|Adventure|Comedy|Crime
3                        Birdcage, The (1996)                         Comedy
4                                Crumb (1994)                    Documentary
